Prepare data

Build the Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(Net, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "lstm":
            self.lstm = nn.LSTM(input_size, hidden_size, n_layers,
                                             batch_first=True, bidirectional=True)
        elif self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        # need crf

    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)   # 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)
        
        x = self.encoder(x.view(1, -1))
        
        # Forward propagate RNN
        if self.model == "lstm":
            out, _ = self.lstm(x, (h0, c0))
        elif self.model == "gru":
            out, _ = self.gru(x, (h0, c0))
        
        output = self.decoder(x.view(1, -1))
        
        return x
    
